# Epidemiology model

https://nbviewer.jupyter.org/github/pyro-ppl/pyro/blob/sir-tutorial-ii/tutorial/source/epi_regional.ipynb?fbclid=IwAR3Gv8tLuiEjOmZh7-NQUa_ggm_QUqtSc5TxRZ0_pSxVA7Y3lWWzSFGKjrA 


In [ ]:
!git clone https://github.com/pyro-ppl/pyro.git

In [ ]:
%cd /content/pyro


In [ ]:
!pip install .[extras]

In [ ]:
import os
import logging
import urllib.request
from collections import OrderedDict

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import pyro
import pyro.distributions as dist
from pyro.contrib.epidemiology import CompartmentalModel, binomial_dist, infection_dist
from pyro.ops.tensor_utils import convolve

%matplotlib inline
pyro.enable_validation(True)           
torch.set_default_dtype(torch.double) 


  ## Model without Policies
  

In [ ]:
class CovidModel(CompartmentalModel):
    def __init__(self, population, new_cases, new_recovered, new_deaths):
        '''
        population (int) – Total population = S + E + I + R.
        '''
        assert len(new_cases) == len(new_recovered) == len(new_deaths)

        compartments = ("S", "E", "I", "D")  # R is implicit.
        duration = len(new_cases)
        super().__init__(compartments, duration, population)

        self.new_cases = new_cases
        self.new_deaths = new_deaths
        self.new_recovered = new_recovered
        

    def global_model(self):
        tau_i = pyro.sample("rec_time", dist.Normal(15.0, 3.0))
        tau_e = pyro.sample("incub_time", dist.Normal(5.0, 1.0))
        # R0 = pyro.sample("R0", dist.LogNormal(0., 1.))
        R0 = pyro.sample("R0", dist.Normal(2.5, 0.5))
        rho = pyro.sample("rho", dist.Beta(10, 10))  # About 50% response rate.
        mort_rate = pyro.sample("mort_rate", dist.Beta(2, 50))  # About 2% mortality rate.
        rec_rate = pyro.sample("rec_rate",dist.Beta(10, 10)) # About 50% recovery rate.
        return R0, tau_e, tau_i, rho, mort_rate, rec_rate

    def initialize(self, params):
        # Start with a single infection.
        return {"S": self.population - 1, "E": 0, "I": 1, "D": 0}

    def transition(self, params, state, t):
        R0, tau_e, tau_i, rho, mort_rate, rec_rate = params

        # Sample flows between compartments.
        S2E = pyro.sample("S2E_{}".format(t),
                            infection_dist(individual_rate=R0 / tau_i,
                                          num_susceptible=state["S"],
                                          num_infectious=state["I"],
                                          population=self.population))
        E2I = pyro.sample("E2I_{}".format(t),
                            binomial_dist(state["E"], 1 / tau_e )) 
        I2R = pyro.sample("I2R_{}".format(t),
                            binomial_dist(state["I"], 1 / tau_i))
        I2D = pyro.sample("I2D_{}".format(t),
                          binomial_dist(state["I"], mort_rate / tau_i))

        # Update compartments with flows.
        state["S"] = state["S"] - S2E 
        state["E"] = state["E"] + S2E - E2I
        state["I"] = state["I"] + E2I - I2R - I2D
        state["D"] = state["D"] + I2D

        # Condition on observations.
        t_is_observed = isinstance(t, slice) or t < self.duration
        pyro.sample("new_cases_{}".format(t),
                    binomial_dist(S2E, rho),
                    obs=self.new_cases[t] if t_is_observed else None)
        pyro.sample("new_deaths_{}".format(t),
                      binomial_dist(I2D, 1),
                      obs=self.new_deaths[t] if t_is_observed else None)
        pyro.sample("new_recovered_{}".format(t),
                    binomial_dist(I2R, rho),
                    obs=self.new_recovered[t] if t_is_observed else None)
        
    def compute_flows(self, prev, curr, t):
        S2E = prev["S"] - curr["S"]  # S can only go to E.
        I2D = curr["D"] - prev["D"]  # D can only have come from I.
        # We deduce the remaining flows by conservation of mass:
        #   curr - prev = inflows - outflows
        E2I = prev["E"] - curr["E"] + S2E
        I2R = prev["I"] - curr["I"] + E2I - I2D
        return {
            "S2E_{}".format(t): S2E,
            "E2I_{}".format(t): E2I,
            "I2D_{}".format(t): I2D,
            "I2R_{}".format(t): I2R,
        }

## Create Country

In [ ]:
# function to make the time series of confirmed and daily confirmed cases for a specific country
def create_country (country, start_date, end_date, state = False) : 

    url = 'https://raw.githubusercontent.com/assemzh/ProbProg-COVID-19/master/full_grouped.csv'
    data = pd.read_csv(url)

    data.Date = pd.to_datetime(data.Date)

    if state :
        df = data.loc[data["Province/State"] == country, ["Province/State", "Date", "Confirmed", "Deaths", "Recovered", "Active", "New cases", "New deaths", "New recovered"]]
    else : 
        df = data.loc[data["Country/Region"] == country, ["Country/Region", "Date", "Confirmed", "Deaths", "Recovered", "Active", "New cases", "New deaths", "New recovered"]]
    df.columns = ["country", "date", "confirmed", "deaths", "recovered", "active", "new_cases", "new_deaths", "new_recovered"]

    # group by country and date
    df = df.groupby(['country','date'])['confirmed', 'deaths', 'recovered',"active", "new_cases", "new_deaths", "new_recovered"].sum().reset_index()

    # convert date string to datetime
    df.date = pd.to_datetime(df.date)
    df = df.sort_values(by = "date")
    df = df[df.date >= start_date]
    df = df[df.date <= end_date]

    active = df['active'].tolist()
    recovered = df['recovered'].tolist()
    deaths = df['deaths'].tolist()
    new_cases = df['new_cases'].tolist()
    new_recovered = df['new_recovered'].tolist()
    new_deaths = df['new_deaths'].tolist()
    
    active = torch.tensor(list(map(float, active))).view(len(active),1) 
    recovered = torch.tensor(list(map(float, recovered))).view(len(recovered),1) 
    deaths = torch.tensor(list(map(float, deaths))).view(len(deaths),1) 
    new_cases = torch.tensor(list(map(float, new_cases))).view(len(new_cases),1)  
    new_recovered = torch.tensor(list(map(float, new_recovered))).view(len(new_recovered),1) 
    new_deaths = torch.tensor(list(map(float, new_deaths))).view(len(new_deaths),1) 


    return_data = {
        'active':active,
        'recovered':recovered,
        'deaths':deaths,
        'new_cases':new_cases,
        'new_recovered': new_recovered,
        'new_deaths':new_deaths }
        
    return return_data


## Get data for countries


In [ ]:
# Parameters
country = "Japan"
start_date = "2020-02-01" 
end_date = "2020-04-01"
population = 126500000


In [ ]:
data = create_country(country, start_date, end_date)

##Train the model using MCMC.



In [ ]:
%%time
model = CovidModel(population, data["new_cases"], data["new_recovered"], data["new_deaths"] )
mcmc = model.fit_mcmc(num_samples=500, warmup_steps = 200)
mcmc.summary()

INFO 	 Running inference...
Sample: 100%|██████████| 700/700 [03:19,  3.51it/s, step size=1.19e-03, acc. prob=0.919]



                     mean       std    median      5.0%     95.0%     n_eff     r_hat
             R0      1.92      0.01      1.92      1.91      1.93      2.54      2.55
 auxiliary[0,0] 126499843.01      0.34 126499842.95 126499842.51 126499843.61      4.48      1.14
 auxiliary[0,1] 126499707.58      0.59 126499707.44 126499706.71 126499708.60      3.22      1.83
 auxiliary[0,2] 126499567.86      3.73 126499567.41 126499562.87 126499573.83      2.58      2.41
 auxiliary[0,3] 126499427.98      6.65 126499426.72 126499417.67 126499438.37      2.63      2.40
 auxiliary[0,4] 126499285.36     10.77 126499282.20 126499270.45 126499303.10      2.78      2.18
 auxiliary[0,5] 126499140.60     16.45 126499134.80 126499119.30 126499168.53      2.75      2.21
 auxiliary[0,6] 126498994.65     21.93 126498988.06 126498965.60 126499030.59      2.76      2.22
 auxiliary[0,7] 126498849.08     27.55 126498841.25 126498812.46 126498893.46      2.75      2.24
 auxiliary[0,8] 126498703.46     32.95 1264